# CSCI E-82A
## Homework assignment 1

> **Note:** To solve the problems in this exercise, compute the probabilities using Bayes' theorem. In subsequent exercises, you will use more computationally efficient and easier to formulate graphical models. 

## Scenario

Inspector Andrea Markov receives a call that Lord Fauntleroy has been murdered in his manor house. She is told that only the cook and butler were present in the house at the time of the crime and that there is no evidence of an intruder. Further, she learns that the only possible murder weapons are a knife and poison. 

Before leaving the police station the Inspector instructs Sargent Bernoulli to gather information on the outcomes of similar murder investigations. She tells the Sargent that it is entirely possible that both the maid and the cook could have worked together. Since the cook and the butler could have acted alone or together, we will assume independence: $p(C,B) = p(C)p(B)$  

From the available data Bernoulli is able to construct the following tables for the probabilities that the cook, $C$, or butler, $B$ is responsible for the murder. 

In [1]:
import pandas as pd
import numpy as np

C = pd.DataFrame({'I':[0.7,0.3]}, index = ['C0','C1'])
print(C.transpose())
print(' ')
B = pd.DataFrame({'I':[0.4,0.6]}, index = ['B0','B1'])
print(B.transpose())

    C0   C1
I  0.7  0.3
 
    B0   B1
I  0.4  0.6


Sargent Bernoulli also discovers that cooks use poison, $P$, or knifes, $K$ with equal probability, whereas butlers favor knifes. Keeping in mind that the suspects could have acted together, he constructs the following table:

In [2]:
W = pd.DataFrame({'P':[0.5,0.3,0.4],'K':[0.5,0.7,0.6]}, index = ['C','B', 'C_B']).transpose()
print(W)



     C    B  C_B
P  0.5  0.3  0.4
K  0.5  0.7  0.6


Given this prior information, Inspector Markov can compute the prior probabilities that the cook or the butler committed the crime, given the type of weapon used. 

In the cell below, compute the prior probabilities that the cook or the butler committed the crime with each of the possible weapons (you may want to add code cells to separate the steps of your calculation). 

> **Hint 1:** Use Bayes' theorem to find the probabilities $p(B|K)$, $p(B|P)$, $p(C|K)$ and $p(C|P)$. Since the suspects may have worked together, the sum of these probabilities will be greater than 1.0.    
> **Hint 2:** Computing the numerators is relatively straight forward. For the denominator there are 8 possible cases. However two cases can be eliminated, since there are only two suspects $P( \urcorner C \wedge  \urcorner B) = 0$, where $ \urcorner$ is the logical not operator and $\wedge$ is the logical and. 

In [3]:
#notation of expansion

# P(B|K) denominator = p(b)*(p(C)*(P(K|B and C)+P(P|B and C)) + P(not C)*(P(K|B and not C)+P(P|B and not C))
#                 + p(not b)*(P(C) *(P(K|not B and C)+P(P|not B and C)))
                           
# P(B|K) numerator = p(b)*(P(C)*P(K|B and C) +P(not C)*P(K|B and not C))    

#notation

# P(B) =    B.loc["B1"]
# P(not B) = B.loc["B0"]
# P(C) = C.loc["C1"]
# P(not C) = C.loc["C0"]

# P(B and C) = B.loc["B1"]*C.loc["C1"] ;0.18
# P(B and not C) = B.loc["B1"]*C.loc["C0"]
# P(not B and C) =B.loc["B0"]*C.loc["C1"]
# P(not B and not C) = 0 ;not use

# P(K|B and C) = W.C_B[1]
# P(P|B and C) = W.C_B[0]
# P(K|B and not C) = W.B[1]
# P(P|B and not C) = W.B[0]
# P(K|not B and C) = W.C[1]
# P(P|not B and C) = W.C[0]

In [4]:
denominator = B.loc["B1"]*(C.loc["C1"] *(W.C_B[1]+W.C_B[0]) + C.loc["C0"]*(W.B[1]+W.B[0]) 
                          ) + B.loc["B0"] * (C.loc["C1"] * (W.C[1] +W.C[0]))

b_given_k_numerator = B.loc["B1"]*(C.loc["C1"]*W.C_B[1] + C.loc["C0"]*W.B[1])
b_given_p_numerator = B.loc["B1"]*(C.loc["C1"]*W.C_B[0] + C.loc["C0"]*W.B[0])
c_given_k_numerator = C.loc["C1"]*(B.loc["B1"]*W.C_B[1] + B.loc["B0"]*W.C[1])
c_given_p_numerator = C.loc["C1"]*(B.loc["B1"]*W.C_B[0] + B.loc["B0"]*W.C[0])

b_given_k = b_given_k_numerator/denominator
b_given_p = b_given_p_numerator/denominator
c_given_k = c_given_k_numerator/denominator
c_given_p = c_given_p_numerator/denominator


What the Inspector really wants to know is the prior probability that each of the suspects in guilty, regardless of the weapon used. In the cell below, compute the probability each suspect is guilty by summing over the possible weapons. 

In [5]:
investigation = np.array([b_given_k,b_given_p,c_given_k,c_given_p])
print(investigation)

[[0.55833333]
 [0.275     ]
 [0.23333333]
 [0.18333333]]


In [6]:
if ((b_given_k+b_given_p).values>(c_given_k+c_given_p).values):
    print("Butler")
else:
    print("Cook")

Butler


From these prior probabilities it seems that the Inspector should focus her investigation on the more likely suspect. Which suspect is that? 

ANS: Butler

## Adding evidence

Upon arriving at the crime scene, Inspector Markov finds that Doctor Turing has already examined the body. The Doctor tells Markov that she is completely convinced that a knife wound is the cause of death, and has found no sign of poisoning. Given this new evidence, the Inspector must update her belief in which suspect is most likely to have committed the crime. 

In the cell below, compute the probabilities that either the butler or the cook has committed the crime. In other words, compute the probabilities for the case where $p(Poison) = 0$.

> **Note:** Once again the probabilities will not add to 1.0 since the suspects may have collaborated in the murder. 

In [7]:
newdenominator = B.loc["B1"]*(C.loc["C1"] *(W.C_B[1]) + C.loc["C0"]*(W.B[1] 
                          ) + B.loc["B0"] * (C.loc["C1"] * (W.C[1])))

In [8]:
newb_given_k = b_given_k_numerator/newdenominator
newc_given_k = c_given_k_numerator/newdenominator
print(newb_given_k.values,newc_given_k.values)

[0.91780822] [0.38356164]


In [9]:
if ((newb_given_k).values>(newc_given_k).values):
    print("Butler")
else:
    print("Cook")

Butler


In the cell below compare the updated probabilities that each suspect is guilty to the prior belief that Inspector Markov had before obtaining the additional evidence. How can you best explain this change?

ANS: Butler(same)